<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Covid-19-India-Analysis" data-toc-modified-id="Covid-19-India-Analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Covid-19 India Analysis</a></span><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Analysis-India-Deaths" data-toc-modified-id="Analysis-India-Deaths-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Analysis India Deaths</a></span><ul class="toc-item"><li><span><a href="#Confirmed-cases-trend" data-toc-modified-id="Confirmed-cases-trend-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Confirmed cases trend</a></span></li><li><span><a href="#Death-trend" data-toc-modified-id="Death-trend-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Death trend</a></span></li><li><span><a href="#Cured" data-toc-modified-id="Cured-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Cured</a></span></li><li><span><a href="#Confirmed,-Deaths-and-Cured" data-toc-modified-id="Confirmed,-Deaths-and-Cured-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>Confirmed, Deaths and Cured</a></span></li><li><span><a href="#State-wise-trend" data-toc-modified-id="State-wise-trend-1.2.5"><span class="toc-item-num">1.2.5&nbsp;&nbsp;</span>State wise trend</a></span><ul class="toc-item"><li><span><a href="#Total-count-as-of-2nd-April" data-toc-modified-id="Total-count-as-of-2nd-April-1.2.5.1"><span class="toc-item-num">1.2.5.1&nbsp;&nbsp;</span>Total count as of 2nd April</a></span></li></ul></li><li><span><a href="#Animation" data-toc-modified-id="Animation-1.2.6"><span class="toc-item-num">1.2.6&nbsp;&nbsp;</span>Animation</a></span></li></ul></li></ul></li></ul></div>

# Covid-19 India Analysis

## Description

Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).[6] The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.[7][8] Common symptoms include fever, cough, and shortness of breath.[9] Other symptoms may include muscle pain, sputum production, diarrhea, sore throat, loss of smell, and abdominal pain.[4][10][11] While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.[7][12] As of 3 April 2020, more than 1,040,000[5] cases of COVID-19 have been reported in more than two hundred countries and territories,[13] resulting in over 55,100 deaths.[5] More than 221,000 people have recovered.[5]

The virus is mainly spread during close contact,[a] and by small droplets produced when people cough, sneeze, or talk.[14][16][15] Respiratory droplets may be produced during breathing but the virus is not generally airborne.[14][16] People may also contract COVID-19 by touching a contaminated surface and then their face.[14][15] It is most contagious when people are symptomatic, although spread may be possible before symptoms appear.[15] The virus can survive on surfaces up to 72 hours.[17] Time from exposure to onset of symptoms is generally between two and fourteen days, with an average of five days.[9][18] The standard method of diagnosis is by reverse transcription polymerase chain reaction (rRT-PCR) from a nasopharyngeal swab.[19] The infection can also be diagnosed from a combination of symptoms, risk factors and a chest CT scan showing features of pneumonia.[20][21]

Recommended measures to prevent infection include frequent hand washing, social distancing (maintaining physical distance from others, especially from those with symptoms), covering coughs and sneezes with a tissue or inner elbow, and keeping unwashed hands away from the face.[22][23] The use of masks is recommended for those who suspect they have the virus and their caregivers.[24] Recommendations for mask use by the general public vary, with some authorities recommending against their use, some recommending their use, and others requiring their use.[25][26][27] Currently, there is no vaccine or specific antiviral treatment for COVID-19. Management involves treatment of symptoms, supportive care, isolation, and experimental measures.[28]

The World Health Organization (WHO) declared the 2019–20 coronavirus outbreak a Public Health Emergency of International Concern (PHEIC)[29][30] on 30 January 2020, and a pandemic on 11 March 2020.[8] Local transmission of the disease has been recorded in many countries across all six WHO regions.[31] 

Source https://en.wikipedia.org/wiki/Coronavirus_disease_2019

## Analysis India Deaths

In [1]:
# import libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Load covid_19_india
covid_19_india_df = pd.read_csv('./data/covid_19_india.csv', index_col='Sno')

In [3]:
covid_19_india_df.Date = pd.to_datetime(covid_19_india_df.Date, format='%d/%m/%y')

In [4]:
covid_19_india_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 587 entries, 1 to 587
Data columns (total 8 columns):
Date                        587 non-null datetime64[ns]
Time                        587 non-null object
State/UnionTerritory        587 non-null object
ConfirmedIndianNational     587 non-null object
ConfirmedForeignNational    587 non-null object
Cured                       587 non-null int64
Deaths                      587 non-null int64
Confirmed                   587 non-null int64
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 41.3+ KB


In [5]:
cm = sns.light_palette("red", as_cmap=True)
covid_19_india_df.groupby(['Date', 'State/UnionTerritory']).sum().style.background_gradient(cmap=cm, axis=0)


,,Cured,Deaths,Confirmed
Date,State/UnionTerritory,,,
2020-01-30 00:00:00,Kerala,0,0,1
2020-01-31 00:00:00,Kerala,0,0,1
2020-02-01 00:00:00,Kerala,0,0,2
2020-02-02 00:00:00,Kerala,0,0,3
2020-02-03 00:00:00,Kerala,0,0,3
2020-02-04 00:00:00,Kerala,0,0,3
2020-02-05 00:00:00,Kerala,0,0,3
2020-02-06 00:00:00,Kerala,0,0,3
2020-02-07 00:00:00,Kerala,0,0,3


In [6]:
date_group = covid_19_india_df.groupby(['Date']).sum()
date_group.reset_index(inplace=True)

In [7]:
date_group.sort_values('Date',inplace=True)

### Confirmed cases trend

In [8]:
import plotly.express as px
fig = px.bar(date_group, x='Date', y='Confirmed', color='Confirmed')

fig.update_layout(
    title="Confirmed cases",    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="black"
    )
)

fig.show()

### Death trend

In [9]:
fig = px.bar(date_group, x='Date', y='Deaths', color='Deaths')

fig.update_layout(
    title="Deaths",    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="black"
    )
)

fig.show()

### Cured

In [10]:
fig = px.bar(date_group, x='Date', y='Cured', color='Cured')

fig.update_layout(
    title="Cured trend",    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="black"
    )
)

fig.show()

### Confirmed, Deaths and Cured

In [11]:
# Create traces
fig = go.Figure()

# Create and style traces
fig.add_trace(go.Scatter(x=date_group.Date, y=date_group.Deaths, name='Deaths',
                         line=dict(color='firebrick', width=2)))
fig.add_trace(go.Scatter(x=date_group.Date, y=date_group.Confirmed, name = 'Confirmed',
                         line=dict(color='royalblue', width=1)))
fig.add_trace(go.Scatter(x=date_group.Date, y=date_group.Cured, name='Cured',
                         line=dict(color='green', width=2,
                              dash='dash') 
))

fig.update_layout(
    title="All Cases",
    xaxis_title="Date",
    yaxis_title="Count",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="black"
    )
)

fig.show()



In [12]:
fig = go.Figure(data=[
    go.Bar(name='Confirmed', x=date_group.Date, y=date_group.Confirmed),
    go.Bar(name='Deaths', x=date_group.Date, y=date_group.Deaths),
    go.Bar(name='Cured', x=date_group.Date, y=date_group.Cured)
])
# Change the bar mode
fig.update_layout(autosize=False,
    width=1000,
    height=750,
    title="All Cases",
    xaxis_title="Date",
    yaxis_title="Count",
    barmode='relative')
fig.show()

### State wise trend

In [13]:
date_table = covid_19_india_df.groupby(['Date', 'State/UnionTerritory']).sum().reset_index().set_index('Date')

#### Total count as of 2nd April

In [14]:
date_table.loc['2020-04-02'].sum()

State/UnionTerritory    Andaman and Nicobar IslandsAndhra PradeshAssam...
Cured                                                                 156
Deaths                                                                 53
Confirmed                                                            2069
dtype: object

In [15]:

fig = go.Figure(data=[
    go.Bar(name='Confirmed', x=date_table.loc['2020-04-02']['State/UnionTerritory'], y=date_table.loc['2020-04-02'].Confirmed),
    go.Bar(name='Deaths', x=date_table.loc['2020-04-02']['State/UnionTerritory'], y=date_table.loc['2020-04-02'].Deaths),
    go.Bar(name='Cured', x=date_table.loc['2020-04-02']['State/UnionTerritory'], y=date_table.loc['2020-04-02'].Cured)
])
# Change the bar mode
fig.update_layout(
    autosize=False,
    width=1000,
    height=750,
    title="State wise cases",
    xaxis_title="State",
    yaxis_title="Count",
    barmode='stack')
fig.show()

In [16]:
fig = px.pie(date_table.loc['2020-04-02'], values='Confirmed', names='State/UnionTerritory', title='Confirmed cases')
fig.update_layout(
    autosize=False,
    width=900,
    height=650)
fig.show()

In [17]:
fig = px.pie(date_table.loc['2020-04-02'], values='Deaths', names='State/UnionTerritory', title='Deaths')
fig.update_layout(
    autosize=False,
    width=900,
    height=650)

fig.show()

In [18]:
fig = px.pie(date_table.loc['2020-04-02'], values='Cured', names='State/UnionTerritory', title='Cured')
fig.update_layout(
    autosize=False,
    width=900,
    height=650)

fig.show()

### Animation

In [174]:
dummy = pd.DataFrame()

dummy['State'] = covid_19_india_df['State/UnionTerritory'].unique()

import plotly.io as pio

pio.templates.default = "simple_white"

def animate_bar_chart():
    frames = []
    grouped = covid_19_india_df[['Date', 'State/UnionTerritory', 'Confirmed', 'Deaths', 'Cured']].groupby(['Date'])
    
    for name, group in iter(grouped):
        merged = pd.merge(group, dummy, how='outer', left_on='State/UnionTerritory', right_on='State')        
        merged.fillna(0, inplace=True)
        merged.sort_values('State', inplace=True)
        frames.append(go.Frame(data = [go.Bar(x = merged['State'].tolist(), y=merged['Confirmed'].tolist())], 
                              layout=go.Layout(title='Confirmed cases - '+group.Date.iloc[0].strftime('%Y-%m-%d'))))
    
    fig = go.Figure(
        data = [go.Bar(x = merged['State'].tolist(), y = [0] * len(merged['State'].tolist()))],
        frames=frames, 
        layout=go.Layout(
            width=1000,
            height=750,
            xaxis=dict(type='category'),
            yaxis=dict(range=[0, 350], autorange=False),
            title="Confirmed cases",
            updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None]),
                         dict(label="Pause",
                              method="skip",                              
                              args=[None])])]))
    fig.show()
                                       
                                       

animate_bar_chart()